In [1]:
!pip install hf_xet
!pip install trl
!pip install --upgrade transformers
!pip install tensorboard
!pip install -U bitsandbytes
!pip install transformers_stream_generator

In [2]:
import os
from datasets import load_dataset, concatenate_datasets, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SMU_MITB_LLM/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SMU_MITB_LLM


In [5]:
# load tuning sets for <Sunrise on the Reaping>
sunrise_train_path = './reasoned_qa_output/sunrise_train_data.json'
sunrise_val_path = './reasoned_qa_output/sunrise_val_data.json'
sunrise_test_path = './reasoned_qa_output/sunrise_test_data.json'

sunrise_train_datasets = Dataset.from_json(sunrise_train_path)
sunrise_val_datasets = Dataset.from_json(sunrise_val_path)
sunrise_test_datasets = Dataset.from_json(sunrise_test_path)

In [6]:
# load tuning sets for <All Fours>
allfours_train_path = './reasoned_qa_output/allfours_train_data.json'
allfours_val_path = './reasoned_qa_output/allfours_val_data.json'
allfours_test_path = './reasoned_qa_output/allfours_test_data.json'

allfours_train_datasets = Dataset.from_json(allfours_train_path)
allfours_val_datasets = Dataset.from_json(allfours_val_path)
allfours_test_datasets = Dataset.from_json(allfours_test_path)

In [7]:
# Combine two books together
train_set = concatenate_datasets([sunrise_train_datasets, allfours_train_datasets])
val_set = concatenate_datasets([sunrise_val_datasets, allfours_val_datasets])
test_set = concatenate_datasets([sunrise_test_datasets, allfours_test_datasets])

In [8]:
train_set

Dataset({
    features: ['Question', 'Answer', 'Reasoning'],
    num_rows: 958
})

In [9]:
val_set

Dataset({
    features: ['Question', 'Answer', 'Reasoning'],
    num_rows: 120
})

In [10]:
test_set

Dataset({
    features: ['Question', 'Answer', 'Reasoning'],
    num_rows: 120
})

In [11]:
print(train_set["Question"][-1])

What trick does Jordi teach the protagonist to help stop thinking about Davey?


In [12]:
from huggingface_hub import notebook_login
notebook_login()

In [14]:
# Load Reasoning LLM
gemma_model = 'google/gemma-2b'
# Load the specified tokenizer for the model to ensure consistency
tokenizer = AutoTokenizer.from_pretrained(gemma_model)

In [15]:
# load model
model = AutoModelForCausalLM.from_pretrained(
    gemma_model,
    device_map="auto"  # instructs the transformers library to automatically distribute the model's layer cross available decives to optimise memory usage
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 19,611,648 || all params: 2,525,784,064 || trainable%: 0.7765


In [17]:
def formatting_prompts_func(examples):
    texts = []
    for question, answer, reasoning_list in zip(examples["Question"], examples["Answer"], examples["Reasoning"]):
        reasoning_str = "\n".join([f"Step {i+1}: {step}" for i, step in enumerate(reasoning_list)])
        formatted_text = (
            f"### Question:\n{question}\n\n"
            f"### Thought Process:\n"
            f"{reasoning_str}\n\n"
            f"### Answer:\n{answer}"
        )
        texts.append(formatted_text)
    return {"text": texts}

In [18]:
train_dataset_formatted = train_set.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=train_set.column_names
)
validation_dataset_formatted = val_set.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=val_set.column_names
)

Map:   0%|          | 0/958 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

In [19]:
print(train_dataset_formatted['text'][-1])

### Question:
What trick does Jordi teach the protagonist to help stop thinking about Davey?

### Thought Process:
Step 1: During their meeting, Jordi demonstrates a technique she used to quit smoking.
Step 2: The protagonist snaps a rubber band on her wrist each time she thinks of Davey.
Step 3: This method aims to reduce obsessive thoughts through punishment.

### Answer:
Jordi teaches her to use a rubber band for negative reinforcement whenever she thinks about him.


In [20]:
import inspect
from trl import SFTTrainer
print(inspect.signature(SFTTrainer.__init__))

(self, model: Union[str, torch.nn.modules.module.Module, transformers.modeling_utils.PreTrainedModel], args: Union[trl.trainer.sft_config.SFTConfig, transformers.training_args.TrainingArguments, NoneType] = None, data_collator: Optional[transformers.data.data_collator.DataCollator] = None, train_dataset: Union[datasets.arrow_dataset.Dataset, datasets.iterable_dataset.IterableDataset, NoneType] = None, eval_dataset: Union[datasets.arrow_dataset.Dataset, dict[str, datasets.arrow_dataset.Dataset], NoneType] = None, processing_class: Union[transformers.tokenization_utils_base.PreTrainedTokenizerBase, transformers.image_processing_utils.BaseImageProcessor, transformers.feature_extraction_utils.FeatureExtractionMixin, transformers.processing_utils.ProcessorMixin, NoneType] = None, compute_loss_func: Optional[Callable] = None, compute_metrics: Optional[Callable[[transformers.trainer_utils.EvalPrediction], dict]] = None, callbacks: Optional[list[transformers.trainer_callback.TrainerCallback]] 

In [25]:

sft_config = SFTConfig(
    output_dir="./results", # From TrainingArguments
    num_train_epochs=3, # From TrainingArguments
    per_device_train_batch_size=4, # From TrainingArguments
    per_device_eval_batch_size=4,
    warmup_steps=500,
    gradient_accumulation_steps=1, # From TrainingArguments
    learning_rate=2e-4, # From TrainingArguments
    logging_steps=10, # From TrainingArguments
    save_steps=500, # From TrainingArguments
    # Add SFT-specific parameters here:
    max_seq_length=128, # Now correctly placed in SFTConfig
    dataset_text_field="text", # Important for SFTTrainer
    packing=True, # Recommended for efficient SFT
    optim="adafactor",
    fp16=True, # For mixed precision, if SFTConfig handles this
)


trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer, # Keep this, as it's separate from args
    train_dataset=train_dataset_formatted,
    eval_dataset=validation_dataset_formatted,
    peft_config=lora_config,
    args=sft_config, # <-- Pass the SFTConfig object here!
    # formatting_func=formatting_func, # If you have a custom formatting function
)

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:404: UserWarning: Padding-free training is enabled, but the attention implementation is not set to 'flash_attention_2'. Padding-free training flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that reliably supports this. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation='flash_attention_2'` in the model configuration, or verify that your attention mechanism can handle flattened sequences.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:450: UserWarning: You are using packing, but the attention implementation is not set to 'flash_attention_2'. Packing flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that re

In [26]:
import torch
torch.cuda.empty_cache()

In [27]:
print("Model Tuning Start...")
trainer.train()
print("Model Tuning Completed")

Model Tuning Start...


Step,Training Loss
10,2.758400
20,2.627800
30,2.585300
40,2.468200
50,2.459300
60,2.369100
70,2.199600
80,2.087200
90,2.039300
100,1.850600


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


Model Tuning Completed


In [28]:
# Save LoAR Adapter
output_model_path = "./LoRA_Gemma"
trainer.save_model(output_model_path)
print(f"Fine-tuned LoRA Adapter saved to: {output_model_path}")

Fine-tuned LoRA Adapter saved to: ./LoRA_Gemma


In [29]:
!pip install rouge_score sacrebleu

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 12.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a113ac6bab023ba65a1e365573999f0599d6cae11d05ddd2a6accd8460a00697
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [30]:
from rouge_score import rouge_scorer
from sacrebleu.metrics import BLEU

In [31]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model_path = "google/gemma-2b"
lora_path = "./LoRA_Gemma"

try:
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        trust_remote_code=True,
        device_map="auto"
    )

    model = PeftModel.from_pretrained(base_model, lora_path)

    tokenizer = AutoTokenizer.from_pretrained(
        base_model_path,
        trust_remote_code=True
    )

    print("LoRA Model Uploaded Successfully")

except Exception as e:
    print(f"LoRA Model Uploaded Failed: {e}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LoRA Model Uploaded Successfully


In [32]:
def formatting_prompts_func(examples):
    texts = []
    for question, answer, reasoning_list in zip(examples["Question"], examples["Answer"], examples["Reasoning"]):
        reasoning_str = "\n".join([f"Step {i+1}: {step}" for i, step in enumerate(reasoning_list)])
        formatted_text = (
            f"### Question:\n{question}\n\n"
            f"### Thought Process:\n"
            f"{reasoning_str}\n\n"
            f"### Answer:\n{answer}"
        )
        texts.append(formatted_text)
    return {"text": texts}

In [33]:
def evaluate_model(model, tokenizer, dataset):
    model.eval()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    bleu = BLEU()

    all_rouge_scores = {'rouge1': {'fmeasure': 0, 'precision': 0, 'recall': 0},
                'rouge2': {'fmeasure': 0, 'precision': 0, 'recall': 0},
                'rougeL': {'fmeasure': 0, 'precision': 0, 'recall': 0}}
    all_bleu_scores = 0
    count = 0

    for example in dataset:
        prompt = example["text"].split("### Answer:")[0] + "### Answer:"
        reference = example["text"].split("### Answer:")[1].strip()

        input_data = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
        input_ids = input_data.input_ids.to(model.device)
        attention_mask = input_data.attention_mask.to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=100,
                num_return_sequences=1,
                pad_token_id=tokenizer.eos_token_id,
                attention_mask = attention_mask
            )

        generated_text = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True).strip()

        rouge_scores = scorer.score(reference, generated_text)
        for key in all_rouge_scores:
            all_rouge_scores[key]['fmeasure'] += rouge_scores[key].fmeasure
            all_rouge_scores[key]['precision'] += rouge_scores[key].precision
            all_rouge_scores[key]['recall'] += rouge_scores[key].recall

        all_bleu_scores += bleu.sentence_score(generated_text, [reference]).score

        count += 1

    avg_rouge_scores = {}
    for key in all_rouge_scores:
        avg_rouge_scores[key] = {metric: score / count for metric, score in all_rouge_scores[key].items()}
    avg_bleu_score = all_bleu_scores / count

    return avg_rouge_scores, avg_bleu_score


In [34]:
test_dataset_formatted = test_set.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=test_set.column_names
)
test_rouge, test_bleu = evaluate_model(model, tokenizer, test_dataset_formatted)
print("Test Set Metrics:")
print("ROUGE Scores:", test_rouge)
print("BLEU Score:", test_bleu)

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Test Set Metrics:
ROUGE Scores: {'rouge1': {'fmeasure': 0.3946784969789285, 'precision': 0.4051854502524122, 'recall': 0.4033796877345059}, 'rouge2': {'fmeasure': 0.21179513131825636, 'precision': 0.21732846287112945, 'recall': 0.215895888797156}, 'rougeL': {'fmeasure': 0.3491536946535655, 'precision': 0.3581925984705251, 'recall': 0.3565503975435979}}
BLEU Score: 16.27422905874661


In [ ]:
torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(
    output_model_path,
    device_map="auto" # Let transformers handle device placement
)

tokenizer = AutoTokenizer.from_pretrained(output_model_path)

# The model should already be on the device because of device_map="auto"
# No need for an explicit model.to(device) call if device_map="auto" is used

def chat_with_model(model, tokenizer):
    print("Start chatting with the model. Type 'quit' to exit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            break

        # Format the user input as a prompt
        prompt = f"### Question:\n{user_input}\n\n### Thought Process:\n"

        # Ensure input_ids are on the correct device
        input_data = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
        input_ids = input_data.input_ids.to(model.device)
        attention_mask = input_data.attention_mask.to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=200, # Adjust based on desired response length
                num_return_sequences=1,
                attention_mask=attention_mask,
                pad_token_id=tokenizer.eos_token_id,
                temperature=0.7, # Adjust for creativity (lower is less creative)
                top_p=0.9,       # Adjust for sampling
                do_sample=True   # Enable sampling
            )

        generated_text = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True).strip()

        # Find the start of the Answer section to only display the answer
        answer_start_index = generated_text.find("### Answer:")
        if answer_start_index != -1:
            # Find the start of the next section (if any)
            next_section_index = generated_text.find("###", answer_start_index + len("### Answer:"))
            if next_section_index != -1:
                generated_answer = generated_text[answer_start_index + len("### Answer:"):next_section_index].strip()
            else:
                generated_answer = generated_text[answer_start_index + len("### Answer:"):].strip()
            print(f"Model: {generated_answer}")
        else:
            # If no explicit Answer section, print the generated text after the Thought Process
            thought_process_end_index = generated_text.find("\n\n### Answer:")
            if thought_process_end_index != -1:
                print(f"Model: {generated_text[:thought_process_end_index].strip()}")
            else:
                 print(f"Model: {generated_text.strip()}")


# Start the chat interface
chat_with_model(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Start chatting with the model. Type 'quit' to exit.
You: Please summarize <Sunrise on the Reaping>


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Model: The chapter begins with a 'clear morning', the Reaping crew 'hurried around' preparing, and the 'sun rises on the reaping'. Haymitch feels 'sick
You: quit


In [ ]:
# 1>
# What items does Haymitch give his mother and brother before leaving for the Games?
# 2>
# What is the name of the plant at Plutarch’s conservatory with a faint sweet rotten smell?
# 3>
# What item does Maysilee use to remove a spike from her cheek?
# 4>
# What is the item Maysilee knuckle-rolls on the train, and what is it made of?
# 5>
# What are the squirrel mutts’ primary movement method and coat color?

In [ ]:
def chat_with_model(model, tokenizer):
    print("Start chatting with the model. Type 'quit' to exit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            break

        # Format the user input as a prompt
        prompt = f"### Question:\n{user_input}\n\n### Thought Process:\n"

        # Ensure input_ids are on the correct device
        input_data = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
        input_ids = input_data.input_ids.to(model.device)
        attention_mask = input_data.attention_mask.to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=200, # Adjust based on desired response length
                num_return_sequences=1,
                attention_mask=attention_mask,
                pad_token_id=tokenizer.eos_token_id,
                temperature=0.7, # Adjust for creativity (lower is less creative)
                top_p=0.9,       # Adjust for sampling
                do_sample=True   # Enable sampling
            )

        generated_text = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True).strip()

        # Find the start of the Answer section to only display the answer
        answer_start_index = generated_text.find("### Answer:")
        if answer_start_index != -1:
            # Find the start of the next section (if any)
            next_section_index = generated_text.find("###", answer_start_index + len("### Answer:"))
            if next_section_index != -1:
                generated_answer = generated_text[answer_start_index + len("### Answer:"):next_section_index].strip()
            else:
                generated_answer = generated_text[answer_start_index + len("### Answer:"):].strip()
            print(f"Model: {generated_answer}")
        else:
            # If no explicit Answer section, print the generated text after the Thought Process
            thought_process_end_index = generated_text.find("\n\n### Answer:")
            if thought_process_end_index != -1:
                print(f"Model: {generated_text[:thought_process_end_index].strip()}")
            else:
                 print(f"Model: {generated_text.strip()}")


# Start the chat interface
chat_with_model(model, tokenizer)

Start chatting with the model. Type 'quit' to exit.
You: What items does Haymitch give his mother and brother before leaving for the Games?


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Model: Haymitch gives his mother 'a gold ring with a tiny carved figure of a swan on it' and his brother 'a pair of scissors, saying “I need them for the Peacekeepers. They’ll cut your tongue out for sure”'.
You: What is the name of the plant at Plutarch’s conservatory with a faint sweet rotten smell?
Model: The plant at Plutarch's conservatory with a 'faint sweet rotten smell' is called 'Daisys'.
You: What item does Maysilee use to remove a spike from her cheek?
Model: Maysilee uses her thumb to remove a spike from her cheek.
You: What is the item Maysilee knuckle-rolls on the train, and what is it made of?
Model: Maysilee knuckle-rolls on a 'piece of metal inlaid with a pattern of tiny stars and moons'.
You: What are the squirrel mutts’ primary movement method and coat color?
Model: The squirrel mutts primarily move by 'bouncing from branch to branch' and have 'gorgeous golden coats'.
You: quit


In [35]:
import json
mcq_data_path = "./reasoned_qa_output/qa_data.json"

# Load the MCQ data
with open(mcq_data_path, 'r') as f:
    mcq_data = json.load(f)
mcq_data

[{'question': "In the novel 'All Fours', what reason does the narrator's neighbor, Brian, give in a note for his concern?",
  'options': {'A': 'He saw someone trying to break into her car.',
   'B': 'He witnessed a stranger using a telephoto lens to photograph her house.',
   'C': 'He found a suspicious package on her doorstep.',
   'D': 'He noticed a coyote lingering in her front yard.'},
  'answer': 'B'},
 {'question': "According to Harris's theory in the book, what is the primary characteristic of a 'Parker'?",
  'options': {'A': 'They enjoy long, uneventful drives and simple pleasures.',
   'B': 'They are cautious and always plan ahead.',
   'C': 'They need to perform impossible tasks and receive applause.',
   'D': 'They are introverted and prefer to stay home.'},
  'answer': 'C'},
 {'question': "What is the narrator's stated motivation for deciding to drive across the country to New York?",
  'options': {'A': "To become a more grounded and 'chill' person, like a 'Driver'.",
   'B

In [36]:
def evaluate_mcq(model, tokenizer, mcq_data):
    model.eval()
    correct_count = 0
    total_count = 0

    for item in mcq_data:
        question = item["question"]
        options = "\n".join([f"{key}. {value}" for key, value in item["options"].items()])
        correct_answer_key = item["answer"]

        prompt = f"### Question:\n{question}\n{options}\n\n### Thought Process:\n"

        input_data = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
        input_ids = input_data.input_ids.to(model.device)
        attention_mask = input_data.attention_mask.to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=200,
                num_return_sequences=1,
                pad_token_id=tokenizer.eos_token_id,
                attention_mask=attention_mask,
                temperature=0.1, # Keep temperature low for deterministic answers,
                do_sample=False  # Disable sampling for deterministic answers
            )

        generated_text = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True).strip()

        # Extract the predicted answer key from the generated text
        # This part might need adjustment based on the exact output format of your model
        # Look for the part after "### Answer:" and try to match it to the option keys (A, B, C, D)
        predicted_answer_key = None
        answer_start_index = generated_text.find("### Answer:")
        if answer_start_index != -1:
            answer_content = generated_text[answer_start_index + len("### Answer:"):].strip()
            # Simple heuristic: look for a single character (like A, B, C, D) at the start
            if len(answer_content) > 0 and answer_content[0] in item["options"].keys():
                predicted_answer_key = answer_content[0]
            elif len(answer_content) > 1 and answer_content[1] == '.' and answer_content[0] in item["options"].keys():
                 predicted_answer_key = answer_content[0]


        print(f"Question: {question}")
        print(f"Predicted Answer Key: {predicted_answer_key}")
        print(f"Correct Answer Key: {correct_answer_key}")


        if predicted_answer_key is not None and predicted_answer_key == correct_answer_key:
            correct_count += 1
            print("Result: Correct")
        else:
            print("Result: Incorrect")

        total_count += 1
        print("-" * 20)


    accuracy = (correct_count / total_count) * 100 if total_count > 0 else 0
    return accuracy

print("\nEvaluating on MCQ data...")
mcq_accuracy = evaluate_mcq(model, tokenizer, mcq_data)
print(f"\nMCQ Accuracy: {mcq_accuracy:.2f}%")

torch.cuda.empty_cache()

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Evaluating on MCQ data...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Question: In the novel 'All Fours', what reason does the narrator's neighbor, Brian, give in a note for his concern?
Predicted Answer Key: B
Correct Answer Key: B
Result: Correct
--------------------


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

Question: After the Games, what does Haymitch see on a wall in a back alley that confirms Lenore Dove's rebellious activities?
Predicted Answer Key: None
Correct Answer Key: C
Result: Incorrect
--------------------


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Question: What does Maysilee use to create a makeshift glue to repair Kerna's broken sunflower token?
Predicted Answer Key: None
Correct Answer Key: B
Result: Incorrect
--------------------


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Question: What is the first muttation Haymitch successfully fights off using his flint striker and a gas plant?
Predicted Answer Key: None
Correct Answer Key: C
Result: Incorrect
--------------------
Question: What is the final 'poster' Haymitch creates in the arena?
Predicted Answer Key: None
Correct Answer Key: D
Result: Incorrect
--------------------

MCQ Accuracy: 4.00%
